# 제주 빅데이터 경진대회

## 1. 라이브러리, 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('201901-202003.csv')
data.head()

REG_YYMM CARD_SIDO_NM CARD_CCG_NM STD_CLSS_NM HOM_SIDO_NM HOM_CCG_NM  AGE  \
0    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  20s   
1    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  30s   
2    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  30s   
3    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  40s   
4    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  40s   

   SEX_CTGO_CD  FLC  CSTMR_CNT      AMT  CNT  
0            1    1          4   311200    4  
1            1    2          7  1374500    8  
2            2    2          6   818700    6  
3            1    3          4  1717000    5  
4            1    4          3  1047300    3

**데이터 설명**

REG__YYMM = 년월

CARD_SIDO_NM = 카드이용지역_시도 (가맹점 주소 기준)

CARD_CCG_NM = 카드이용지역_시군구 (가맹점 주소 기준)

STD_CLSS_NM = 업종명

HOM_SIDO_NM = 거주지역_시도 (고객 집주소 기준)

HOM_CCG_NM = 거주지역_시군구 (고객 집주소 기준)

AGE = 연령대

SEX_CTGO_CD = 성별 (1: 남성, 2: 여성)

FLC = 가구생애주기 (1: 1인가구, 2: 영유아자녀가구, 3: 중고생자녀가구, 4: 성인자녀가구, 5: 노년가구)

CSTMR_CNT = 이용고객수 (명)

AMT = 이용금액 (원)

CNT = 이용건수 (건)

## 2. 데이터 전처리

In [3]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [4]:
# 날짜 처리
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [5]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [6]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

## 3. 탐색적 자료분석

## 4. 변수 선택 및 모델 구축

In [7]:
from sklearn.model_selection import train_test_split , GridSearchCV

# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['AMT','CSTMR_CNT','CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

X_train, X_test, y_train, y_test = train_test_split(train_features, train_target, test_size=0.2, random_state=0)

## 5. 모델 학습 및 검증

#### 1) GridSearchCV를 이용하여 최적 파라미터 값 찾기

In [28]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, GridSearchCV
lgbm_Card = LGBMRegressor()

In [29]:
params = {
            'learning_rate' : [0.075,0.05,0.1],
            'num_iterations' : [1000],
            'max_depth' : [15,20,25], 
            'boosting_type': ['dart','gbdt'],
            'feature_fraction': [0.7,0.8,0.9],
            'scale_pos_weight': [1.1,1.3,1.5]
        }
gcv = GridSearchCV(lgbm_Card, param_grid = params,cv=KFold(3), scoring='neg_mean_squared_error', n_jobs=-1)
gcv.fit(train_features,train_target)
print('final params', gcv.best_params_)  
print('best score', gcv.best_score_)

final params {'boosting_type': 'gbdt', 'feature_fraction': 0.9, 'learning_rate': 0.1, 'max_depth': 15, 'num_iterations': 1000, 'scale_pos_weight': 1.1}
best score -1.0994616026370574


#### 2) 찾은 파라미터 값을 넣어서 예측 모형 구축

In [59]:
train_ds = lgb.Dataset(X_train, label=y_train)
val_ds = lgb.Dataset(X_test, label=y_test)
params = {
            'learning_rate' : 0.1,
            'feature_fraction' : 1,
            'max_depth' : -1, 
            'scale_pos_weight' : 1,
            'boosting_type': 'gbdt',
            'metric': 'rmse',
        }

In [61]:
model = lgb.train(params,
                  train_ds,
                  20000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.78403
[200]	valid_0's rmse: 1.58093
[300]	valid_0's rmse: 1.44084
[400]	valid_0's rmse: 1.33721
[500]	valid_0's rmse: 1.24717
[600]	valid_0's rmse: 1.18567
[700]	valid_0's rmse: 1.14043
[800]	valid_0's rmse: 1.09929
[900]	valid_0's rmse: 1.06965
[1000]	valid_0's rmse: 1.03672
[1100]	valid_0's rmse: 1.01154
[1200]	valid_0's rmse: 0.989875
[1300]	valid_0's rmse: 0.970103
[1400]	valid_0's rmse: 0.946305
[1500]	valid_0's rmse: 0.928463
[1600]	valid_0's rmse: 0.914475
[1700]	valid_0's rmse: 0.902132
[1800]	valid_0's rmse: 0.889396
[1900]	valid_0's rmse: 0.878399
[2000]	valid_0's rmse: 0.869951
[2100]	valid_0's rmse: 0.858356
[2200]	valid_0's rmse: 0.848853
[2300]	valid_0's rmse: 0.840386
[2400]	valid_0's rmse: 0.833415
[2500]	valid_0's rmse: 0.827514
[2600]	valid_0's rmse: 0.821547
[2700]	valid_0's rmse: 0.815727
[2800]	valid_0's rmse: 0.808651
[2900]	valid_0's rmse: 0.80334
[3000]	valid_0's rmse: 0.797953

## 6. 결과 및 결언

In [62]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [63]:
# LightGBM 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [64]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [65]:
# 제출 파일 만들기
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

REG_YYMM CARD_SIDO_NM           STD_CLSS_NM           AMT
id                                                           
0     202004           강원            건강보조식품 소매업  1.675570e+08
1     202004           강원               골프장 운영업  5.884980e+09
2     202004           강원           과실 및 채소 소매업  1.488769e+09
3     202004           강원     관광 민예품 및 선물용품 소매업  2.898655e+07
4     202004           강원  그외 기타 분류안된 오락관련 서비스업  3.060250e+06